# 1. Introduction

## CFPB Consumer Complaints EDA

## 📦 About the Dataset

The Consumer Financial Protection Bureau (CFPB) collects and publishes thousands of consumer complaints each week regarding financial products and services.  
Each complaint is sent to the relevant company, and responses are tracked. By making this data public, the CFPB aims to improve transparency and accountability in the financial marketplace.

- **Source:** [CFPB Consumer Complaints Database](https://www.consumerfinance.gov/data-research/consumer-complaints/)
- **Data columns:**  
    - Date received, product, sub-product, issue, sub-issue, company, state, response, etc.
    - Includes text columns, categorical columns, and timestamps
- **Typical size:** 2M+ rows

---

## 🎯 Project Goal

**To analyze the landscape of consumer complaints in the U.S. financial sector and extract actionable insights for companies, regulators, and consumers.**

### Main Objectives
- Understand trends in complaints across time, products, and companies
- Identify major issues and pain points faced by consumers
- Assess company response rates and behaviors

---

## 🛠️ What We Do (Methods)

- **Data Cleaning & Preprocessing**
    - Handling missing/null values
    - Standardizing product and issue names
    - Parsing dates and categoricals
- **Exploratory Data Analysis (EDA)**
    - Trends over time: volume of complaints per month/year
    - Breakdown by product, sub-product, issue, state, and company
    - Distribution of company responses (timeliness, public statements)
    - Visualization: bar charts, time series, heatmaps, etc.
- **Business Insights**
    - Detect which products/companies have the most issues
    - Find common root causes of complaints
    - Compare top-performing and underperforming companies

---

## 🔍 Key Questions & Insights

- Which financial products generate the most consumer complaints?
- What are the most common issues and sub-issues?
- Which companies have the highest volume of complaints?
- How do response rates and response types differ across companies?
- Are there geographic (state-level) patterns in complaint volume or issues?
- Has the complaint volume increased/decreased over time?

---

## 📊 Possible KPIs (Key Performance Indicators)

- **Total Complaints per Product/Company**
- **Percentage of Timely Responses**
- **Average Response Time** (if timestamp data is available)
- **Complaint Resolution Rate** (if status available)
- **Top Issues by Frequency**
- **Null/Missing Data Rate** per critical column

---

## 💡 Expected Outcomes

- **Actionable recommendations** for financial companies to improve service
- **Transparency** for consumers on which companies/products have most complaints
- **Regulatory insights** to help prioritize oversight and policy

---

## 📝 Author

Minhyeok Son  
[LinkedIn](hwww.linkedin.com/in/minhyeokson) | [GitHub](https://github.com/Shawn-Son)

---

# 2.Data Loading

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df = pd.read_csv('../../Datasets/finance/consumer_complaints.csv')

/var/folders/vs/dy5n5y354mz0k83z44gbr5ph0000gn/T/ipykernel_45028/182790902.py:1: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../Datasets/finance/consumer_complaints.csv')


# 3.

In [ ]:
df.head()